In [ ]:
# Program to allocate scarce resources
# import libraries
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

### TO DO
Fix filter_portal_df - make sure canceled orders aren't being included.
Include most recent order date in active df. Min date?
Add qty on ready to
Rearrange columns
Append Ready To orders to Active
Add header
Reduce repetitive code

In [ ]:
# return the date 6 months + 1 day ago. Use to filter completed record.
six_months_ago = datetime.now() + relativedelta(months=-6, days=-1)
print(six_months_ago)

In [ ]:
# open CSVs and convert to pandas dataframes
dir_downloads = 'C:\\Users\\Purchasing_Kelsey\\Downloads\\'
dir_taq = 'C:\\Users\\Purchasing_Kelsey\\Documents\\K\'s Projects\\Taq Triage\\'
dir_taq_records = 'C:\\Users\\Purchasing_Kelsey\\Documents\\K\'s Projects\\Taq Triage\\Taq Records\\'

files = [
        'Active_Orders (14).csv',
        'Ready_to_Order (19).csv',
        'Completed (7).csv', # Past 6 months order history
        'Taq_Summary_2024.csv'
]
try:
        df_active = pd.read_csv(dir_downloads + files[0], index_col=None, header = 0)
        df_ready_to = pd.read_csv(dir_downloads + files[1], index_col=None, header = 0)
        df_complete_6mo = pd.read_csv(dir_downloads + files[2], index_col=None, header = 0)
        df_summary_2024 = pd.read_csv(dir_taq_records + files[3], index_col=None, header = 0)

except Exception as e:
        print(e)

In [ ]:
# Convert date requested to datetime
df_active['Date requested'] = pd.to_datetime(df_active['Date requested'])
df_ready_to['Date requested'] = pd.to_datetime(df_ready_to['Date requested'])
df_complete_6mo['Date requested'] = pd.to_datetime(df_complete_6mo['Date requested'])

In [ ]:
# Check filtering. To do - streamline. Fix min date check.
print('df_active:', df_active['Status'].unique(), df_active['Vendor Requested'].unique(), df_active['Catalog No'].unique(), df_active['Status'].unique())
print('df_ready_to:', df_ready_to['Status'].unique(), df_ready_to['Vendor Requested'].unique(), df_ready_to['Catalog No'].unique(), df_ready_to['Status'].unique())
print('df_complete_6mo:', df_complete_6mo['Status'].unique(), df_complete_6mo['Vendor Requested'].unique(), df_complete_6mo['Catalog No'].unique(), df_complete_6mo['Status'].unique(), min(df_complete_6mo['Date requested']))

In [ ]:
# Define functions

# function to filter dfs to keep only target items. Accepts a df and lists of vendors, cat nos, statsuses, and column names.
def filter_portal_df(df, keep_vend, keep_cat, rm_stat):
    try:
        # Filter based on the provided lists
        df = df[df['Vendor Requested'].isin(keep_vend)].copy()
        df = df[df['Catalog No'].isin(keep_cat)].copy()
        df = df[~df['Status'].isin(rm_stat)].copy()
        
        # Debugging: Print unique values for verification
        print('Vendors: ', df['Vendor Requested'].unique())
        print('Cat Nos: ', df['Catalog No'].unique())
        print('Statuses: ', df['Status'].unique())

        return df.copy()
    except Exception as e:
        print(e)

# function to drop columns that aren't keys in a dict
def drop_cols(df, keep_col_dict): 
    keep_col_list = list(keep_col_dict.keys())
    drop_col = df.columns[~df.columns.isin(keep_col_list)]
    df = df.drop(df[drop_col], axis=1)
    return df.copy()

def get_5_recent_orders(df):
    try:
        df_sorted = df.sort_values(by=['Lab ID', 'Date requested'], ascending=[True, False])

        df_rec_ord = df_sorted.groupby('Lab ID').head(5)
        df_rec_ord.head()
        return df_rec_ord

    except Exception as e:
        print(e)

# Function to filter df_completed to contain only orders for labs with pending Taq orders.
# Accepts filtered Active Orders & filtered completed orders dfs
def keep_target_labIDs(df_act, df_comp):
    target_labs = df_act['Lab ID'].unique()
    df_comp = df_comp[df_comp['Lab ID'].isin(target_labs)]
    return df_comp.copy()

# function to save to a given directory with a given name
def save_to_csv(df, save_dir, save_name):
    save_name = save_name + datetime.today().strftime('%m.%d.%Y') + ".csv"
    try:
        df.to_csv(save_dir + save_name, encoding='utf-8', index=False)
        print(save_name, "saved")
        
    except Exception as e:
        print(e)

# Updated drop cols function
def drop_cols_2(df, rm_col_list): 
    drop_col = df.columns[df.columns.isin(rm_col_list)]
    df = df.drop(df[drop_col], axis=1)
    return df.copy()

In [ ]:
# define lists of values to keep and exclude
keep_catnos = ['T-1012']

rm_statuses = ['Canceled']

keep_vendors = [
    'DCW',
    'DCW-K',
    'DCWK',
    'IEH Equipment'
]

keep_col_portal = {
    'Date requested':'datetime',
    'Lab ID':str,
    'Vendor Requested':str,
    'Product Name/Description':str,
    'Catalog Number Requested':str,
    'Quantity':int,
    'Date Order Placed':'datetime',
    'Tracking #':str,
    'sheet':str,
}


In [ ]:
#Filter Active df
df_active = filter_portal_df(df_active, keep_vendors, keep_catnos, rm_statuses)
df_active = drop_cols(df_active, keep_col_portal)
df_active.head()

#save_to_csv(df_active, dir_taq, 'TaqActive_')

In [ ]:
# Filter Ready To
df_ready_to = filter_portal_df(df_ready_to, keep_vendors, keep_catnos, rm_statuses)
df_ready_to = drop_cols(df_ready_to, keep_col_portal)
df_ready_to.head()

# save_to_csv(df_ready_to, dir_taq, 'TaqReadyTo_')

In [ ]:
# filter df_complete_6mo
df_complete_6mo = drop_cols(df_complete_6mo, keep_col_portal)
#save_to_csv(df_complete_6mo, dir_taq, 'TaqCompleted6mo_')
df_complete_6mo.head()

In [ ]:
# Make a summary column
df_active_total_qty = df_active[['Lab ID', 'Quantity']].copy()
df_active_total_qty = df_active_total_qty.groupby('Lab ID').sum()
df_active_total_qty.head()

df_active_oldest_date = df_active[['Lab ID', 'Date requested']].copy()
df_active_oldest_date = df_active.groupby('Lab ID')['Date requested'].min()
df_active_oldest_date.head()

df_active_summary = pd.merge(df_active_total_qty, df_active_oldest_date, how = 'left', on='Lab ID')
df_active_summary['sheet'] = 'Active'

df_active_summary.head()

In [ ]:
df_ready_to_total_qty = df_ready_to[['Lab ID', 'Quantity']].copy()
df_ready_to_total_qty = df_ready_to_total_qty.groupby('Lab ID').sum()
df_ready_to_total_qty.head()

df_ready_to_oldest_date = df_ready_to[['Lab ID', 'Date requested']].copy()
df_ready_to_oldest_date = df_ready_to_oldest_date.groupby('Lab ID')['Date requested'].min()
df_ready_to_oldest_date.head()
df_ready_to_summary = pd.merge(df_ready_to_total_qty, df_ready_to_oldest_date, how = 'left', on='Lab ID')
df_ready_to_summary['sheet'] = 'Ready to'

In [ ]:
# append summary dfs to each other
df_pending_summary = pd.concat([df_active_summary, df_ready_to_summary])
df_pending_summary.head()

In [ ]:
# Get 6 Month Summary
df_6mo_summary = df_complete_6mo[['Lab ID', 'Quantity']].copy()
df_6mo_summary = df_6mo_summary.groupby('Lab ID').sum()

df_6mo_summary.head()

In [ ]:
# merge dfs
df_merge = df_pending_summary.merge(df_summary_2024, how = 'left', on = 'Lab ID')
df_merge = df_merge.merge(df_6mo_summary, how = 'left', on = 'Lab ID')
df_merge.head()

df_merge = df_merge.rename(columns={'Quantity_x':'Qty', 'Quantity_y':'6mo Total'})\

df_merge.head()

In [ ]:
# calculated columns
df_merge['% Filled'] = df_merge['6mo Total']/(df_merge['Total Qty 2024']/2)

df_merge = df_merge.sort_values(by='% Filled')
df_merge.head()

In [ ]:
# make final df
df_final = df_merge[['Lab ID','sheet', 'Monthly Avg', '% Filled', 'Qty', ]].copy()
df_final['Send Qty'] = ''
df_final['Lab Emailed?'] = ''

df_final.head()

In [ ]:
save_to_csv(df_final, dir_taq, 'Summary_')